In [23]:
from instance_reader import read_instance
from heuristics import greedy_wave_selection
from Low_levels import LowLevel1

# instancia

In [24]:
instance = read_instance("datasets/a/instance_0002.txt")

In [25]:
# printeamos las ordenes
print("Instance:", instance)
for i, order in enumerate(instance.orders):
    print(f"Order {i + 1}: {order}")

    # printeamos los pasillos
print("Instance:", instance)
for i, runner in enumerate(instance.runners):
    print(f"Order {i + 1}: {runner}")

Instance: Instance with 7 orders and 33 runners, covering 7 item
Order 1: Order 0: {2: 3} (Total units: 3)
Order 2: Order 1: {0: 2} (Total units: 2)
Order 3: Order 2: {3: 1} (Total units: 1)
Order 4: Order 3: {1: 1} (Total units: 1)
Order 5: Order 4: {4: 2} (Total units: 2)
Order 6: Order 5: {5: 1} (Total units: 1)
Order 7: Order 6: {6: 1} (Total units: 1)
Instance: Instance with 7 orders and 33 runners, covering 7 item
Order 1: Runner 0: {0: 12, 2: 20}, Total units: 32)
Order 2: Runner 1: {6: 8}, Total units: 8)
Order 3: Runner 2: {4: 14}, Total units: 14)
Order 4: Runner 3: {2: 48}, Total units: 48)
Order 5: Runner 4: {6: 12}, Total units: 12)
Order 6: Runner 5: {6: 16}, Total units: 16)
Order 7: Runner 6: {6: 4}, Total units: 4)
Order 8: Runner 7: {0: 8}, Total units: 8)
Order 9: Runner 8: {3: 88}, Total units: 88)
Order 10: Runner 9: {6: 19}, Total units: 19)
Order 11: Runner 10: {0: 12, 1: 98, 2: 16}, Total units: 126)
Order 12: Runner 11: {6: 1}, Total units: 1)
Order 13: Runner 

# solucion

In [26]:
solucion = instance.constructora()
print(solucion)

for orden in solucion.selected_orders:
    print(orden.index, orden.items)

print("------------------")
for runner in solucion.selected_runners:
    print(runner.index, runner.stock)


La solución escoje las ordenes (0,) con un total de 3 unidades y los corredores (10,) con un valor objetivo de 3.00.
0 {2: 3}
------------------
10 {0: 12, 1: 98, 2: 16}


# Low Levels

In [27]:
low_level = LowLevel1(id=1, nombre= "LowLevel1")
# ejecutamos la implementacion del low level hasta que no se pueda mejorar mas
obj_val = 0
while True:
    solucion_antigua = solucion
    solucion = low_level.implementacion(solucion_antigua=solucion)
    if solucion.objective_value == obj_val:
        break
    obj_val = solucion.objective_value
print("Final solution:")
print(solucion.objective_value)


Final solution:
3.0


In [28]:
print(solucion.total_units)
print(solucion.num_runners)

3
1
